In [1]:
# created on Dec 24, 2020
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

!python -m pip install tweepy
import tweepy, json, time

In [2]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.f = open(file, 'a', encoding="utf-8")
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                # {'limit': {'track': 13, 'timestamp_ms': '1585851016736'}}

                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    record = '%s, %s, %f, %f, %s \n' % (username, created_at, lng, lat, text)
                    print(record)
                    self.f.write(record)
                else:
                    pass
        else:
            self.f.close()
            print("finished")
            return False

In [3]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = "assets/tweets.csv"

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "iJnxXRvLBptjmCRvPNhoQRquO"
    consumer_secret = "9NCStOQMhwlb9hSZ0fBv5UufuxsdXyR5Q5Z6UXOvLpkbg10xny"
    access_token = "1349111958121836544-5DNRXT8heNNRaj8Xhf1u0AjwZSu240"
    access_token_secret = "PX7yzClKpOLwj4lOCy501UcNXlRRsGBfYw9U62MIrsIdw"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,  # Contiguous US
                 -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
                 -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii

    stream_listener = StreamListener(time_limit=60, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS)
    stream.filter(track=['president'], is_async=True)

{'created_at': 'Wed Jan 20 23:58:26 +0000 2021', 'id': 1352042837995614214, 'id_str': '1352042837995614214', 'text': '@ZekesMommasKid @Name90001 @nytimes Im just a regular person in Boston media.\nWBCN 14y QUINCY\nCity of Presidents\nWe… https://t.co/VNwXhqFAv0', 'display_text_range': [36, 140], 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': True, 'in_reply_to_status_id': 1352042701894647811, 'in_reply_to_status_id_str': '1352042701894647811', 'in_reply_to_user_id': 1310534796758654977, 'in_reply_to_user_id_str': '1310534796758654977', 'in_reply_to_screen_name': 'JoeshowA', 'user': {'id': 1310534796758654977, 'id_str': '1310534796758654977', 'name': 'JoeShow AirPhone1', 'screen_name': 'JoeshowA', 'location': 'Quincy, MA', 'url': 'http://AirPhone1.com', 'description': 'OWNER: The FakeNewZ\nPG1 AMERICAN REVOLUTION\nBoston massacre. 3/5/1770\n2/24/17-PRESENT\nCyberStationUSA 2009-Present (part owner)\nWBCN BOSTON 1989-2003', 

{'created_at': 'Wed Jan 20 23:58:27 +0000 2021', 'id': 1352042840021467142, 'id_str': '1352042840021467142', 'text': 'Yes she does...we are grateful for her leadership', 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 66206748, 'id_str': '66206748', 'name': 'Bernice Hall', 'screen_name': 'Libslady', 'location': 'Almost Heaven WV via L.I. N.Y.', 'url': None, 'description': 'Well-behaved women seldom make history !  Retired Educator.  Sports, Health, & Fitness Fanatic  #Centrist  #Moderate 💚💙☃️ #AmyForAmerica !', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 1236, 'friends_count': 845, 'listed_count': 21, 'favourites_count': 46290, 'statuses_count': 62137, 'created_at': 'Sun Aug 16 22:20:36 +0000 2009', 'utc

{'created_at': 'Wed Jan 20 23:58:27 +0000 2021', 'id': 1352042841237843968, 'id_str': '1352042841237843968', 'text': '@Concerned4Can @AHousefather @JustinTrudeau Also, travel restrictions have been put in place since early on and hav… https://t.co/6t8zwutuGP', 'display_text_range': [44, 140], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': True, 'in_reply_to_status_id': 1352006504359071744, 'in_reply_to_status_id_str': '1352006504359071744', 'in_reply_to_user_id': 1226183528695697408, 'in_reply_to_user_id_str': '1226183528695697408', 'in_reply_to_screen_name': 'Concerned4Can', 'user': {'id': 1140090941195325440, 'id_str': '1140090941195325440', 'name': 'Ryan Turnbull', 'screen_name': 'TurnbullWhitby', 'location': 'Whitby, Ontario', 'url': 'https://www.facebook.com/TurnbullWhitby', 'description': 'Member of Parliament for Whitby. Father, husband, social entrepreneur and former CEO. 📞905-665-8182 📧Ryan.Turnbull@parl.gc.ca he/hi

{'created_at': 'Wed Jan 20 23:58:27 +0000 2021', 'id': 1352042842261250048, 'id_str': '1352042842261250048', 'text': '@sydbizzles Oh honeyyyyy', 'display_text_range': [12, 24], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': 1352042661243461632, 'in_reply_to_status_id_str': '1352042661243461632', 'in_reply_to_user_id': 217895994, 'in_reply_to_user_id_str': '217895994', 'in_reply_to_screen_name': 'sydbizzles', 'user': {'id': 121979959, 'id_str': '121979959', 'name': 'Shell Bellé✨', 'screen_name': 'ShelbyLeiqh_', 'location': 'Atlanta, GA', 'url': None, 'description': 'I’m a bunch of different characters. literally.', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 2090, 'friends_count': 913, 'listed_count': 9, 'favourites_count': 6119, 'statuses_count': 109917, 'created_at': 'Thu Mar 11 06:14:10 +0000 2010', 'utc_offset': None, 'time_zone': None, 'geo_enabled'

{'created_at': 'Wed Jan 20 23:58:28 +0000 2021', 'id': 1352042844127719431, 'id_str': '1352042844127719431', 'text': '@NoTatsClassy Absolutely not!', 'display_text_range': [14, 29], 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': 1352001538760318976, 'in_reply_to_status_id_str': '1352001538760318976', 'in_reply_to_user_id': 4105765714, 'in_reply_to_user_id_str': '4105765714', 'in_reply_to_screen_name': 'NoTatsClassy', 'user': {'id': 160631338, 'id_str': '160631338', 'name': 'Patricia Bellomo, Author', 'screen_name': 'patriciabellomo', 'location': 'Detroit, MI', 'url': 'http://www.patriciabellomo.com', 'description': 'Novelist. #mafiaromance #mafiathriller / #MAGA. Proud Patriot. Trump 2020.', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 2696, 'friends_count': 2556, 'listed_count': 31, 'favourites_count': 199086, 'statuses_count': 111081, 'created_at': '

...

{'created_at': 'Wed Jan 20 23:59:25 +0000 2021', 'id': 1352043081638395905, 'id_str': '1352043081638395905', 'text': 'Glute game strong https://t.co/00PUVJMMof', 'display_text_range': [0, 17], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 244837655, 'id_str': '244837655', 'name': 'Gianni B', 'screen_name': 'gbiancooo', 'location': None, 'url': 'https://blacklivesmatters.carrd.co/', 'description': 'Insta: @ _Giannibianco. Italian. Fitness. NASM Certified Personal Trainer.', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 2544, 'friends_count': 922, 'listed_count': 3, 'favourites_count': 41064, 'statuses_count': 49329, 'created_at': 'Sun Jan 30 07:25:40 +0000 2011', 'utc_offset': None, 'time_zone': None, 'geo_

{'created_at': 'Wed Jan 20 23:59:25 +0000 2021', 'id': 1352043082875871233, 'id_str': '1352043082875871233', 'text': 'Everyone at Epcot https://t.co/dp3yWaiFd7', 'display_text_range': [0, 17], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1142196787568140289, 'id_str': '1142196787568140289', 'name': 'Aveon', 'screen_name': 'aveonhoward_', 'location': None, 'url': None, 'description': 'performer 🎭 . theme parks🎢 . uotm 🌍💙', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 721, 'friends_count': 225, 'listed_count': 2, 'favourites_count': 15058, 'statuses_count': 5803, 'created_at': 'Fri Jun 21 22:25:09 +0000 2019', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_enabled':

{'created_at': 'Wed Jan 20 23:59:25 +0000 2021', 'id': 1352043085329436675, 'id_str': '1352043085329436675', 'text': '@_mgml88 Das hueva 🤧', 'display_text_range': [9, 20], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': 1351999970400890880, 'in_reply_to_status_id_str': '1351999970400890880', 'in_reply_to_user_id': 761543181108326400, 'in_reply_to_user_id_str': '761543181108326400', 'in_reply_to_screen_name': '_mgml88', 'user': {'id': 362902861, 'id_str': '362902861', 'name': 'E. Beltran⚡️♠️', 'screen_name': 'Beltran19jl', 'location': 'Santa Barbara, CA', 'url': 'https://www.instagram.com/invites/contact/?i=u2m9lypzblk7&utm_content=x8rmm', 'description': 'A veces finjo no saber nada, para así saber hasta dónde mienten🍀📝', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 1794, 'friends_count': 1195, 'listed_count': 2, 'favourites_count': 3934, 'statuses_count':

{'created_at': 'Wed Jan 20 23:59:26 +0000 2021', 'id': 1352043087158075392, 'id_str': '1352043087158075392', 'text': "I'm discussing “Leading in 2021:  #Clarity, #Connection &amp; Conversation” with @MohamedUnfltrd, @Over40Wisdom, Steve… https://t.co/gzShc1AHtW", 'display_text_range': [0, 140], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': True, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 17542227, 'id_str': '17542227', 'name': 'Amy Blaschka', 'screen_name': 'amyblaschka', 'location': 'Pleasanton, CA', 'url': 'http://linktr.ee/amyblaschka', 'description': '#SocialMedia #Ghostwriter for leaders📲👻✍🏼 (Yep, that’s a thing.) @Forbes Contributor👩\u200d💻 ✨ILLUMINATE ME ✨newsletter📥 👇🏼', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 3486, 'friends_count': 1814, 'l

finished
finished
